In [1]:
import $ivy.`org.scala-lang:scala-library:2.13.12`

// Add Delta Lake dependency
import $ivy.`io.delta:delta-spark_2.13:3.1.0`

// Add Spark SQL dependency
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.1`
import $ivy.`org.apache.spark:spark-core_2.13:3.5.1`

//hadoop hdfs
import $ivy.`org.apache.hadoop:hadoop-common:3.3.6`
import $ivy.`org.apache.hadoop:hadoop-client:3.3.6`
import $ivy.`org.apache.hadoop:hadoop-client-api:3.3.6`
import $ivy.`org.apache.hadoop:hadoop-hdfs:3.3.6`

import $ivy.$                                     

// Add Delta Lake dependency

import $ivy.$                                

// Add Spark SQL dependency

import $ivy.$                                      

import $ivy.$                                       

//hadoop hdfs

import $ivy.$                                      

import $ivy.$                                      

import $ivy.$                                          

import $ivy.$                                    


In [2]:
import io.delta.tables.DeltaTable
import org.apache.spark.sql.{Encoders, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import io.delta.tables.DeltaTable

import org.apache.spark.sql.{Encoders, SparkSession}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


In [8]:
val spark=SparkSession.builder()
    .appName("Fetch Delta Table")
    .master("local[*]")//runs on localhost 4040
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000")//setup for connecting to hadoop, comment it out if not required
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.databricks.delta.autoCompact.enabled", "true") // Enable auto-compaction globally. By default will use 128 MB as the target file size.
    .config("spark.databricks.delta.optimizeWrite.enabled", "true")
    .config("spark.databricks.delta.autoCompact.targetFileSize", "256MB") // Set target file size for auto-compaction
    .config("spark.databricks.delta.optimizeWrite.binSize", "256MB") // Set bin size for optimized writes
    .getOrCreate()

24/07/16 17:21:53 INFO SparkContext: Running Spark version 3.5.1
24/07/16 17:21:53 INFO SparkContext: OS info Linux, 6.8.0-38-generic, amd64
24/07/16 17:21:53 INFO SparkContext: Java version 11.0.11
24/07/16 17:21:53 INFO ResourceUtils: ==============================================================
24/07/16 17:21:53 INFO ResourceUtils: No custom resources configured for spark.driver.
24/07/16 17:21:53 INFO ResourceUtils: ==============================================================
24/07/16 17:21:53 INFO SparkContext: Submitted application: Fetch Delta Table
24/07/16 17:21:53 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/07/16 17:21:53 INFO ResourceProfile: Limiting resource is cpu
24/07/16 17:21:53 INFO ResourceProfileManager: Add

spark: SparkSession = org.apache.spark.sql.SparkSession@a5e44be

In [4]:
val delPath="hdfs://localhost:9000/delta/dataF"
val delPath2="hdfs://localhost:9000/delta/dataFsms"
val delPath3="hdfs://localhost:9000/delta/dataFesewa"
val delPath4="/home/avyuthan-shah/Desktop/Data/dataF"

delPath: String = "hdfs://localhost:9000/delta/dataF"
delPath2: String = "hdfs://localhost:9000/delta/dataFsms"
delPath3: String = "hdfs://localhost:9000/delta/dataFesewa"
delPath4: String = "/home/avyuthan-shah/Desktop/Data/dataF"

<h2>Z-Order</h2>

In [ ]:
writeFile("true")//to monitor load
val startTime = System.nanoTime()
val dt=DeltaTable.forPath(spark, delPath)
dt.optimize().executeZOrderBy("AccountNo")
val endTime = System.nanoTime()
writeFile("false")
val elapsedTime = (endTime - startTime) / 1e9 // Time in seconds
println(s"Elapsed time for Zordering: '$elapsedTime' ")

In [13]:
val version=DeltaTable.forPath(spark, delPath).history(1).select("version").as[Long](Encoders.scalaLong).head //Get Latest Version

val startTime = System.nanoTime()
spark.read.format("delta").option("versionAsOf",version).load(delPath).createOrReplaceTempView("delta_table")

val result = spark.sql(
   s"""
   |SELECT * FROM delta_table
   |WHERE AccountNo IN ("56135","69310","65260")
   |ORDER BY AccountNo,VALUEDATE DESC,TIME DESC;
   """.stripMargin)
    
result.show(truncate=false)
val endTime = System.nanoTime()

val elapsedTime = (endTime - startTime) / 1e9 // Time in seconds
println(s"Elapsed time for retrival: '$elapsedTime' ")

24/07/16 17:23:20 INFO BlockManagerInfo: Removed broadcast_20_piece0 on 10.13.164.84:43067 in memory (size: 9.8 KiB, free: 1048.6 MiB)
24/07/16 17:23:20 INFO BlockManagerInfo: Removed broadcast_19_piece0 on 10.13.164.84:43067 in memory (size: 52.8 KiB, free: 1048.7 MiB)
24/07/16 17:23:21 INFO SparkContext: Starting job: getHistory at DeltaTableOperations.scala:56
24/07/16 17:23:21 INFO DAGScheduler: Got job 14 (getHistory at DeltaTableOperations.scala:56) with 8 output partitions
24/07/16 17:23:21 INFO DAGScheduler: Final stage: ResultStage 18 (getHistory at DeltaTableOperations.scala:56)
24/07/16 17:23:21 INFO DAGScheduler: Parents of final stage: List()
24/07/16 17:23:21 INFO DAGScheduler: Missing parents: List()
24/07/16 17:23:21 INFO DAGScheduler: Submitting ResultStage 18 (MapPartitionsRDD[81] at getHistory at DeltaTableOperations.scala:56), which has no missing parents
24/07/16 17:23:21 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 219.4 KiB, fre

+---------+----------+------------------+------+----------+-------------+----------+----------+--------+
|AccountNo|DATE      |TRANSACTIONDETAILS|CHQNO |VALUEDATE |WITHDRAWALAMT|DEPOSITAMT|BALANCEAMT|TIME    |
+---------+----------+------------------+------+----------+-------------+----------+----------+--------+
|56135    |2024-07-08|Deposit           |Null  |2024-07-08|0.0          |460.73    |540.97    |08:04:46|
|56135    |2024-07-05|Withdraw          |Null  |2024-07-05|311.09       |0.0       |-311.09   |07:49:14|
|56135    |2024-06-30|Deposit           |Null  |2024-06-30|0.0          |414.66    |679.31    |18:42:49|
|56135    |2024-06-29|Withdraw          |Null  |2024-06-29|233.21       |0.0       |253.03    |20:57:27|
|56135    |2024-06-26|Withdraw          |Null  |2024-06-26|488.67       |0.0       |-488.67   |16:01:36|
|56135    |2024-06-26|Deposit           |Null  |2024-06-26|0.0          |319.96    |319.96    |15:19:03|
|56135    |2024-06-24|Withdraw          |Null  |2024-06

version: Long = 3L
startTime: Long = 1573904278892L
result: Dataset[Row] = [AccountNo: string, DATE: date ... 7 more fields]
endTime: Long = 1576963514361L
elapsedTime: Double = 3.059235469

In [17]:
val version=DeltaTable.forPath(spark, delPath).history(1).select("version").as[Long](Encoders.scalaLong).head //Get Latest Version

val startTime = System.nanoTime()

spark.read.format("delta").option("versionAsOf",version).load(delPath).createOrReplaceTempView("delta_table")
val result2=spark.sql(
   s"""
   |SELECT AccountNo,SUM(WITHDRAWALAMT) AS TotalWithdrawal,SUM(DEPOSITAMT) AS TotalDeposit FROM delta_table
   |GROUP BY AccountNo;;
   """.stripMargin)

result2.show(truncate=false)

val endTime = System.nanoTime()

val elapsedTime = (endTime - startTime) / 1e9 // Time in seconds
println(s"Elapsed time for retrival: '$elapsedTime' ")

24/07/16 17:25:09 INFO BlockManagerInfo: Removed broadcast_41_piece0 on 10.13.164.84:43067 in memory (size: 18.7 KiB, free: 1048.7 MiB)


24/07/16 17:25:10 INFO SparkContext: Starting job: getHistory at DeltaTableOperations.scala:56
24/07/16 17:25:10 INFO DAGScheduler: Got job 31 (getHistory at DeltaTableOperations.scala:56) with 8 output partitions
24/07/16 17:25:10 INFO DAGScheduler: Final stage: ResultStage 45 (getHistory at DeltaTableOperations.scala:56)
24/07/16 17:25:10 INFO DAGScheduler: Parents of final stage: List()
24/07/16 17:25:10 INFO DAGScheduler: Missing parents: List()
24/07/16 17:25:10 INFO DAGScheduler: Submitting ResultStage 45 (MapPartitionsRDD[149] at getHistory at DeltaTableOperations.scala:56), which has no missing parents
24/07/16 17:25:10 INFO MemoryStore: Block broadcast_42 stored as values in memory (estimated size 219.4 KiB, free 1047.9 MiB)
24/07/16 17:25:10 INFO MemoryStore: Block broadcast_42_piece0 stored as bytes in memory (estimated size 71.8 KiB, free 1047.8 MiB)
24/07/16 17:25:10 INFO BlockManagerInfo: Added broadcast_42_piece0 in memory on 10.13.164.84:43067 (size: 71.8 KiB, free: 104

+---------+------------------+------------------+
|AccountNo|TotalWithdrawal   |TotalDeposit      |
+---------+------------------+------------------+
|14157    |7423.780000000001 |6635.24           |
|65548    |8188.899999999998 |6713.100000000002 |
|21452    |8460.609999999999 |9832.13           |
|14887    |6588.36           |7004.969999999999 |
|69460    |9199.309999999998 |5611.58           |
|57608    |7175.25           |8990.7            |
|87626    |7804.710000000002 |8765.109999999999 |
|56662    |4944.470000000001 |5979.400000000001 |
|82208    |6342.46           |5215.84           |
|35844    |7331.36           |6337.23           |
|56315    |9896.56           |9885.19           |
|43462    |7726.040000000001 |7437.950000000001 |
|60733    |2533.8            |6856.6            |
|77303    |7861.290000000001 |7763.32           |
|51063    |4058.5200000000004|5318.82           |
|98725    |11877.29          |12552.440000000002|
|10351    |15712.73          |12292.179999999997|


version: Long = 3L
startTime: Long = 1682868989242L
result2: Dataset[Row] = [AccountNo: string, TotalWithdrawal: double ... 1 more field]
endTime: Long = 1688184499118L
elapsedTime: Double = 5.315509876

In [6]:
spark.stop()

24/07/16 17:21:36 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/07/16 17:21:36 INFO SparkUI: Stopped Spark web UI at http://10.13.164.84:4040
24/07/16 17:21:36 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/07/16 17:21:36 INFO MemoryStore: MemoryStore cleared
24/07/16 17:21:36 INFO BlockManager: BlockManager stopped
24/07/16 17:21:36 INFO BlockManagerMaster: BlockManagerMaster stopped
24/07/16 17:21:36 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/07/16 17:21:36 INFO SparkContext: Successfully stopped SparkContext
